### Задание 1.

In [1]:
# Открываем файл и читаем его содержимое
with open('./data/sequences.fasta', 'r') as file:
    lines = file.readlines()

# Списки для хранения обработанных данных
vocab_aa = set()  # Словарь аминокислот
histone_types = set()  # Типы гистонов

current_histone_type = None

# Обрабатываем каждую строку
for line in lines:
    line = line.strip()
    if line.startswith('>'):  # Это заголовок последовательности
        # Извлекаем тип гистона
        parts = line.split('|')
        if len(parts) > 2:
            histone_type = parts[2]
            histone_types.add(histone_type)
    else:
        # Это строка с последовательностью
        vocab_aa.update(line)

# Теперь создаём единый словарь
vocab = vocab_aa.union(histone_types)

# Если нужно, можно преобразовать в обычное множество или список
vocab = sorted(vocab)

# Выводим результат
print("Объединённый словарь:")
print(vocab)

Объединённый словарь:
['-', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'H1', 'H2A', 'H2B', 'H3', 'H4', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'X', 'Y']


> Сколько элементов получилось в переменной vocab?

In [2]:
len(vocab)

28

### Задание 2.


In [3]:
import torch

# 1. Создаем char_to_idx и idx_to_char
char_to_idx = {char: idx for idx, char in enumerate(vocab)}
idx_to_char = {idx: char for char, idx in char_to_idx.items()}

# 2. Перечитываем файл для извлечения последовательностей и типов гистонов
sequences = []
histone_types_list = []

with open('./data/sequences.fasta', 'r') as f:
    lines = f.readlines()

i = 0
while i < len(lines):
    line = lines[i].strip()
    if line.startswith('>'):
        # Извлекаем тип гистона
        parts = line.split('|')
        histone_type = parts[2]
        histone_types_list.append(histone_type)

        # Переходим к строке с последовательностью
        i += 1
        if i < len(lines):
            seq = lines[i].strip()
            sequences.append(seq)
    else:
        i += 1

# 3. Кодируем последовательности и типы гистонов
encoded_sequences = []
for seq in sequences:
    encoded_seq = [char_to_idx[c] for c in seq]
    encoded_sequences.append(torch.tensor(encoded_seq))

encoded_types = torch.tensor([char_to_idx[t] for t in histone_types_list])

# 4. Результаты
num_sequences = len(encoded_sequences)
print(f"Количество последовательностей в датасете: {num_sequences}")

# encoded_sequences - список тензоров (последовательности)
# encoded_types - тензор с закодированными типами гистонов

Количество последовательностей в датасете: 565


### Задание 3

Для выполнения задания вы можете самостоятельно создать Python-ноутбук, а в LMS прикладывать лишь требуемые результаты. В этом задании вам необходимо написать рекуррентную нейросеть с механизмом внимания с использованием PyTorch. В качестве входного слоя создайте Embedding. Далее создайте 1 рекуррентный слой. В качестве слоя внимания используйте линейную трансформацию. В качестве выходного слоя используйте линейную трансформацию. Используйте код для решения данной задачи:

In [4]:
import torch
import torch.nn as nn

embedding_dim = 5
hidden_dim = 9

class RNNWithAttentionModel(nn.Module):
   def __init__(self, random_seed=5):
       super(RNNWithAttentionModel, self).__init__()
       torch.manual_seed(random_seed)
       torch.cuda.manual_seed(random_seed)
       torch.backends.cudnn.deterministic = True
       torch.backends.cudnn.benchmark = False
       # Create an embedding layer for the vocabulary
       # your code here
       # Create an RNN layer
       # your code here
       # Apply a linear transformation to get the attention scores
       # your code here
       self.fc = nn.Linear(hidden_dim, vocab_size)
   def forward(self, x):
       x = self.embeddings(x)
       out, _ = self.rnn(x)
       attention_out = self.attention(out).squeeze(2)
       #  Get the attention weights
       # your code here
       # Compute the context vector
       # your code here
       out = self.fc(context)
       return out


In [5]:
import torch
import torch.nn as nn

# Определяем параметры
embedding_dim = 5
hidden_dim = 9
vocab_size = len(vocab)  # Количество уникальных символов из словаря

class RNNWithAttentionModel(nn.Module):
    def __init__(self, random_seed=5):
        super(RNNWithAttentionModel, self).__init__()
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

        # Создаем слой встраивания для словаря
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)

        # Создаем RNN слой
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)

        # Применяем линейную трансформацию для получения оценок внимания
        self.attention = nn.Linear(hidden_dim, 1)

        # Выходной слой
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        x = self.embeddings(x)
        out, _ = self.rnn(x)
        
        # Получаем оценки внимания
        attention_scores = self.attention(out)  # размер [batch_size, seq_len, 1]
        
        # Применяем softmax для вычисления весов внимания
        attention_weights = torch.softmax(attention_scores, dim=1)  # размер [batch_size, seq_len, 1]
        
        # Вычисляем контекстный вектор
        context = torch.bmm(attention_weights.transpose(1, 2), out)  # размер [batch_size, 1, hidden_dim]
        context = context.squeeze(1)  # размер [batch_size, hidden_dim]

        # Получаем выход
        out = self.fc(context)
        return out

In [7]:
import torch
import torch.optim as optim
import torch.nn.functional as F

# Эмуляция входных данных для обучения
# Здесь должны использоваться ваши подготовленные данные
# sequences и encoded_types, как указано ранее
num_epochs = 700
learning_rate = 0.01

# Инициализация модели, функции потерь и оптимизатора
model = RNNWithAttentionModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Обучение модели
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    total_correct = 0
    total_samples = 0

    for seq, histone in zip(encoded_sequences, encoded_types):
        optimizer.zero_grad()  # Обнуляем градиенты

        # Прямой проход
        outputs = model(seq.unsqueeze(0))  # добавляем размерность батча
        
        # Вычисляем потери
        loss = criterion(outputs, histone.unsqueeze(0))  # добавляем размерность батча
        total_loss += loss.item()

        # Рассчитываем количество правильных предсказаний
        _, predicted = torch.max(outputs, 1)  # получаем индексы максимальных значений
        total_correct += (predicted == histone.unsqueeze(0)).sum().item()
        
        # Обновляем общее количество образцов
        total_samples += 1  # Каждый вход является одним образцом

        # Обратный проход и оптимизация
        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(encoded_sequences)
    accuracy = total_correct / total_samples * 100  # вычисляем точность в процентах
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")

print("Обучение завершено.")

Epoch [1/700], Loss: 0.7368, Accuracy: 87.26%
Epoch [2/700], Loss: 0.5342, Accuracy: 88.67%
Epoch [3/700], Loss: 0.5197, Accuracy: 87.79%
Epoch [4/700], Loss: 0.6719, Accuracy: 85.31%
Epoch [5/700], Loss: 0.7351, Accuracy: 84.96%
Epoch [6/700], Loss: 0.7354, Accuracy: 84.60%
Epoch [7/700], Loss: 0.7284, Accuracy: 84.42%
Epoch [8/700], Loss: 0.7204, Accuracy: 84.42%
Epoch [9/700], Loss: 0.7144, Accuracy: 84.42%
Epoch [10/700], Loss: 0.7165, Accuracy: 84.07%
Epoch [11/700], Loss: 0.6842, Accuracy: 84.42%
Epoch [12/700], Loss: 0.6040, Accuracy: 85.84%
Epoch [13/700], Loss: 0.5488, Accuracy: 86.02%
Epoch [14/700], Loss: 0.6122, Accuracy: 86.73%
Epoch [15/700], Loss: 0.7358, Accuracy: 84.25%
Epoch [16/700], Loss: 0.5830, Accuracy: 85.84%
Epoch [17/700], Loss: 0.6582, Accuracy: 86.02%
Epoch [18/700], Loss: 0.5645, Accuracy: 86.90%
Epoch [19/700], Loss: 0.6413, Accuracy: 85.66%
Epoch [20/700], Loss: 0.6629, Accuracy: 86.55%
Epoch [21/700], Loss: 0.6013, Accuracy: 86.19%
Epoch [22/700], Loss: 

In [10]:
# Входная последовательность
seq_fasta = '''>Pan|XP_003311177.1|HTYPE|HVARIANT 

MSGRGKQGGKARTKAKTRSSRAGLQFPVGRVHRLLRKGNYAERVGAGAPVYLAAVLEYLT 

AEILELAGNAARDNKKTRIIPRHLQLAIRNDEELNKLLGKVTIAQGGVLPNIQAVLLPKK 

TESHHKAKGK'''

# Извлекаем последовательность, убирая заголовок и пробелы
sequence_lines = seq_fasta.split('\n')[1:]  # пропускаем заголовок
sequence = ''.join(line.strip() for line in sequence_lines if line.strip())

# Закодируем последовательность в индексы
encoded_sequence = torch.tensor([char_to_idx[c] for c in sequence], dtype=torch.long).unsqueeze(0)  # добавляем размерность батча

# Сделаем предсказание
model.eval()  # Устанавливаем модель в режим оценки
with torch.no_grad():  # Отключаем градиенты
    outputs = model(encoded_sequence)
    _, predicted = torch.max(outputs, 1)  # получаем индексы максимальных значений

# Преобразуем индексы обратно в типы гистонов
predicted_histone_type = idx_to_char[predicted.item()]  # получаем тип гистона по индексу

print("Предсказанный тип гистона:", predicted_histone_type)

Предсказанный тип гистона: H1
